In [1]:
import paho.mqtt.client as mqtt
from optparse import OptionParser
import time
import gc

In [2]:
client_id = '3310-u6142160'

client_name = '3310student'
client_password = 'comp3310'

server_addr = '3310exp.hopto.org'

topic_list = ['counter/slow/']
qos_list   = [0, 1, 2]
def get_topic(topic_list, qos_list):
    return []

print('\n'.join(topic_list))

sys_topic = '$SYS/#'
sys_key   = 'SYS'

counter/slow/


In [13]:
def subscribe_counter(client, userdata, speed, qos_list):
    topic    = 'counter/' + speed + '/q'
    for qos in qos_list:
        cur_topic = topic + str(qos)
        client.subscribe(cur_topic, qos=qos)
        userdata[cur_topic] = []    

# client's event on connect
def on_connect(client, userdata, flags, rc):
    '''
        flags['session present'] = 1 => last session retained by broker, otherwise a new session

        rc 0=Connection successful, 1=incorrect protocol version, 2=invalid client identifier,
           3=server unavailable,    4=bad username or password,   5=not authorised

        userdata: created along with Client class initiliasation
    '''

    # subscribe topic slow
    subscribe_counter(client, userdata, 'slow', [2])
    
#     # subscribe topic fast
#     subscribe_counter(client, userdata, 'fast')
    
#     # subscribe sys
#     client.subscribe(sys_topic, qos=2)
#     userdata[sys_key] = {}
    
    print('flags=', flags, 'rc=', rc, 'userdata=', userdata)

# client's event on subscribe
def on_subscribe(client, userdata, mid, granted_qos):
    print('mid=', mid, 'grated qos=', granted_qos)

# client's event on receiving message
def on_message(client, userdata, msg):
#     if msg.topic.startswith(sys_topic.rstrip('#')): # sys info
#         handle_sys_info(userdata[sys_key], msg)
#     else:
#         try:
#             cnt = int(msg.payload)
#             userdata[msg.topic].append(cnt)
#         except:
#             cnt = str(msg.payload)
    
    userdata[msg.topic].append(msg.payload)
    
    if len(userdata[msg.topic]) > 2:
        client.loop_stop()
        client.disconnect()
    
# client's event on disconnect
def on_disconnect(client, userdata, rc):
    if rc != 0:
        print("Unexpected disconnection.")
    print('log:\n', userdata)

########################################################
    
def handle_sys_info(sys_record, msg):
    return
#     print(msg.topic, msg.payload)

def publish_result(userdata):
    # t 10-minute intervals under
    # studentreport/<your.Uni.ID>/ with the ‘retain’ flag set and QoS=2.
    
    topic = 'studentreport/u6142160'
    qos   = 2
    rt_f  = True
    sub_topic_list = ['/language', '/timestamp']
    
    time_stamp = str(time.time())
    sys_record = userdata['SYS']
    for key in sys_record:
        asd

In [14]:
# initialisation
client = mqtt.Client(client_id, clean_session=True, userdata={}, transport="tcp")
client.username_pw_set(username=client_name, password=client_password)

client.on_connect    = on_connect
client.on_subscribe  = on_subscribe
client.on_message    = on_message
client.on_disconnect = on_disconnect

In [18]:
# potentially blocking => connect_async for non-blocking
client.connect(host=server_addr, port=1883, keepalive=60, bind_address="")
# keepalive: 
#     maximum period in seconds allowed between communications with the broker. 
#     If no other messages are being exchanged, this controls the rate at which the client will send 
#     ping messages to the broker

gc.collect()
client.loop_start()

flags= {'session present': 0} rc= 0 userdata= {'counter/slow/q2': []}
mid= 2 grated qos= (2,)
log:
 {'counter/slow/q2': [b'close', b'914', b'915']}


In [17]:
# disconnect
client.loop_stop()
client.disconnect()
gc.collect()

log:
 {'counter/slow/q2': [b'close', b'880', b'881']}


0

In [ ]:
client.publish(sys_topic.rstrip('#'), payload=client_id, qos=0, retain=False)